In [20]:
import numpy as np 
import pandas as pd 

# To consume Twitter's API
import tweepy
from tweepy import OAuthHandler 


from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [21]:
# keys and tokens from the Twitter Dev Console
consumer_key = 'Sec3MvclRIx2RVlgu9l0SJX6D'
consumer_secret = 'ayoPNWtBm7fWpMBoK6EwRmegu3SW8Rw9mzJkottkv97quPe941'
access_token = '736550752760406018-so5CPJrEbJKb3c3Pq8va3VFr0yk4S0E'
access_token_secret = 'Cgr8tz0h6FTU7kxAjDzpHnjffNTHxWsBytXnu4Ihd1TFb'

In [22]:
class TwitterClient(object): 
    def __init__(self): 
        #Initialization method. 
        try: 
            # create OAuthHandler object 
            auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            # add hyper parameter 'proxy' if executing from behind proxy "proxy='http://172.22.218.218:8085'"
            self.api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
            
        except tweepy.TweepError as e:
            print(f"Error: Tweeter Authentication Failed - \n{str(e)}")

    def get_tweets(self, query, maxTweets = 1000):
        #Function to fetch tweets. 
        # empty list to store parsed tweets 
        tweets = [] 
        sinceId = None
        max_id = -1
        tweetCount = 0
        tweetsPerQry = 100

        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry)
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1))
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break

                for tweet in new_tweets:
                    parsed_tweet = {} 
                    parsed_tweet['tweets'] = tweet.text 

                    # appending parsed tweet to tweets list 
                    if tweet.retweet_count > 0: 
                        # if tweet has retweets, ensure that it is appended only once 
                        if parsed_tweet not in tweets: 
                            tweets.append(parsed_tweet) 
                    else: 
                        tweets.append(parsed_tweet) 
                        
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                # Just exit if any error
                print("Tweepy error : " + str(e))
                break
        
        return pd.DataFrame(tweets)

## (A). Mining Latest topic of Elon Musk from twitter

In [23]:
twitter_client = TwitterClient()

# calling function to get tweets
tweets_df = twitter_client.get_tweets('ElonMusk', maxTweets=100000)
print(f'tweets_df Shape - {tweets_df.shape}')
tweets_df.head(10)

Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 600 tweets
Downloaded 700 tweets
Downloaded 800 tweets
Downloaded 900 tweets
Downloaded 1000 tweets
Downloaded 1100 tweets
Downloaded 1200 tweets
Downloaded 1300 tweets
Downloaded 1400 tweets
Downloaded 1500 tweets
Downloaded 1600 tweets
Downloaded 1700 tweets
Downloaded 1800 tweets
Downloaded 1900 tweets
Downloaded 2000 tweets
Downloaded 2100 tweets
Downloaded 2200 tweets
Downloaded 2300 tweets
Downloaded 2400 tweets
Downloaded 2500 tweets
Downloaded 2600 tweets
Downloaded 2700 tweets
Downloaded 2800 tweets
Downloaded 2900 tweets
Downloaded 3000 tweets
Downloaded 3100 tweets
Downloaded 3200 tweets
Downloaded 3300 tweets
Downloaded 3400 tweets
Downloaded 3500 tweets
Downloaded 3600 tweets
Downloaded 3700 tweets
Downloaded 3800 tweets
Downloaded 3900 tweets
Downloaded 4000 tweets
Downloaded 4100 tweets
Downloaded 4200 tweets
Downloaded 4300 tweets
Downloaded 4400 twee

Rate limit reached. Sleeping for: 216


Downloaded 14906 tweets
Downloaded 15006 tweets
Downloaded 15106 tweets
Downloaded 15206 tweets
Downloaded 15306 tweets
Downloaded 15406 tweets
Downloaded 15506 tweets
Downloaded 15606 tweets
Downloaded 15706 tweets
Downloaded 15806 tweets
Downloaded 15906 tweets
Downloaded 16006 tweets
Downloaded 16106 tweets
Downloaded 16206 tweets
Downloaded 16306 tweets
Downloaded 16406 tweets
Downloaded 16506 tweets
Downloaded 16606 tweets
Downloaded 16706 tweets
Downloaded 16806 tweets
Downloaded 16906 tweets
Downloaded 17006 tweets
Downloaded 17106 tweets
Downloaded 17206 tweets
Downloaded 17306 tweets
Downloaded 17406 tweets
Downloaded 17506 tweets
Downloaded 17606 tweets
Downloaded 17706 tweets
Downloaded 17806 tweets
Downloaded 17906 tweets
Downloaded 18006 tweets
Downloaded 18106 tweets
Downloaded 18206 tweets
Downloaded 18306 tweets
Downloaded 18406 tweets
Downloaded 18506 tweets
Downloaded 18605 tweets
Downloaded 18704 tweets
Downloaded 18796 tweets
Downloaded 18886 tweets
Downloaded 18984

Rate limit reached. Sleeping for: 208


Downloaded 32845 tweets
Downloaded 32945 tweets
Downloaded 33045 tweets
Downloaded 33145 tweets
Downloaded 33245 tweets
Downloaded 33345 tweets
Downloaded 33445 tweets
Tweepy error : Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out.
tweets_df Shape - (25757, 1)


,tweets
0,@elonmusk @realfuckingnews @business Ancient c...
1,"@elonmusk @rockstardenied @SpaceX Very kind, a..."
2,@elonmusk What will happen to Dodge?
3,RT @_0121Dan: Day 8 of tweeting @elonmusk and ...
4,@rockstardenied @elonmusk @SpaceX Download the...
5,@elonmusk I need to buy tesla model x help urg...
6,#ElonMusk: Owning #bitcoin only slightly bette...
7,@elonmusk Who are they?
8,"Congrats Perseverance!!\nHoping to see ""mars r..."
9,@rockstardenied @elonmusk @SpaceX Download the...


## (B) Sentiment Analysis of Elon Musk Tweets

In [28]:
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

In [29]:
sentiments_using_SIA = tweets_df.tweets.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))
pd.DataFrame(sentiments_using_SIA.value_counts())

,tweets
pos,22725
neg,3032


In [30]:
tweets_df['sentiment'] = sentiments_using_SIA
tweets_df.head()

,tweets,sentiment
0,@elonmusk @realfuckingnews @business Ancient c...,pos
1,"@elonmusk @rockstardenied @SpaceX Very kind, a...",pos
2,@elonmusk What will happen to Dodge?,pos
3,RT @_0121Dan: Day 8 of tweeting @elonmusk and ...,neg
4,@rockstardenied @elonmusk @SpaceX Download the...,pos


In [31]:
df = tweets_df
df.to_csv('Elon Musk.csv') 

In [33]:
em = pd.read_csv("Elon Musk.csv")

In [34]:
em.shape

(25757, 3)

In [36]:
em.head(10)

,Unnamed: 0,tweets,sentiment
0,0,@elonmusk @realfuckingnews @business Ancient c...,pos
1,1,"@elonmusk @rockstardenied @SpaceX Very kind, a...",pos
2,2,@elonmusk What will happen to Dodge?,pos
3,3,RT @_0121Dan: Day 8 of tweeting @elonmusk and ...,neg
4,4,@rockstardenied @elonmusk @SpaceX Download the...,pos
5,5,@elonmusk I need to buy tesla model x help urg...,pos
6,6,#ElonMusk: Owning #bitcoin only slightly bette...,pos
7,7,@elonmusk Who are they?,pos
8,8,"Congrats Perseverance!!\nHoping to see ""mars r...",pos
9,9,@rockstardenied @elonmusk @SpaceX Download the...,pos


In [37]:
em.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25757 entries, 0 to 25756
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  25757 non-null  int64 
 1   tweets      25757 non-null  object
 2   sentiment   25757 non-null  object
dtypes: int64(1), object(2)
memory usage: 603.8+ KB
